# Model Calibration

<a href="https://colab.research.google.com/github/thomasjpfan/ml-workshop-intermediate-2-of-2/blob/master/notebooks/02-model-calibration.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

In [ ]:
# Install dependencies for google colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    %pip install -r https://raw.githubusercontent.com/thomasjpfan/ml-workshop-intermediate-2-of-2/master/requirements.txt

In [ ]:
import sklearn
assert sklearn.__version__.startswith("1.0"), "Plese install scikit-learn 1.0"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['savefig.bbox'] = 'tight'

sklearn.set_config(display='diagram')

## Create dummy dataset

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=5000, n_features=20,
                           n_informative=4, n_redundant=4)

train_samples = 1000  # Samples used for training the models

X_train = X[:train_samples]
X_test = X[train_samples:]
y_train = y[:train_samples]
y_test = y[train_samples:]

### Train linear model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

lr = make_pipeline(StandardScaler(), LogisticRegression(random_state=42))
lr.fit(X_train, y_train)

## Calibration curve

In [ ]:
from sklearn.calibration import calibration_curve

In [ ]:
lr_proba = lr.predict_proba(X_test)

In [ ]:
prob_true, prod_pred = calibration_curve(y_test, lr_proba[:, 1], n_bins=5)

print(prob_true)
print(prod_pred)

In [ ]:
from sklearn.calibration import CalibrationDisplay

In [ ]:
CalibrationDisplay.from_estimator(lr, X_test, y_test, n_bins=5)

In [ ]:
from sklearn.metrics import brier_score_loss

In [ ]:
lr_brier = brier_score_loss(y_test, lr_proba[:, 1])

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 8))
CalibrationDisplay.from_estimator(lr, X_test, y_test, n_bins=5, ax=ax1)
ax1.set_title("n_bins=5")
CalibrationDisplay.from_estimator(lr, X_test, y_test, n_bins=10, ax=ax2)
ax2.set_title("n_bins=10")
CalibrationDisplay.from_estimator(lr, X_test, y_test, n_bins=30, ax=ax3)
ax3.set_title("n_bins=30")

### Train Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

In [ ]:
rf_proba = rf.predict_proba(X_test)

In [ ]:
rf_brier = brier_score_loss(y_test, rf_proba[:, 1])
rf_brier

### Train Single Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train, y_train)

In [ ]:
tree_proba = tree.predict_proba(X_test)

In [ ]:
tree_brier = brier_score_loss(y_test, tree_proba[:, 1])
tree_brier

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 8))
CalibrationDisplay.from_estimator(lr, X_test, y_test, n_bins=10, ax=ax1)
ax1.set_title(f"LogisticRegression: {lr_brier:0.4f}")
CalibrationDisplay.from_estimator(tree, X_test, y_test, n_bins=10, ax=ax2)
ax2.set_title(f"DecisionTreeClassifier: {tree_brier:0.4f}")
CalibrationDisplay.from_estimator(rf, X_test, y_test, n_bins=10, ax=ax3)
ax3.set_title(f"RandomForestClassifier: {rf_brier:0.4f}");

## Exercise 1

1. Train a `GradientBoostingClassifier` on the training set.
2. Compute the brier score on the test set for the `GradientBoostingClassifier`.
3. Plot the calibration curve with `n_bins=10`.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

**If you are running locally**, you can uncomment the following cell to load the solution into the cell. On **Google Colab**, [see solution here](https://github.com/thomasjpfan/ml-workshop-intermediate-2-of-2/blob/master/notebooks/solutions/02-ex01-solutions.py).

In [ ]:
# %load solutions/02-ex01-solutions.py

## Calibration

In [ ]:
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
inner_rf = RandomForestClassifier(random_state=0)
cal_rf = CalibratedClassifierCV(inner_rf, method="isotonic")
cal_rf.fit(X_train, y_train)

In [ ]:
cal_rf_proba = cal_rf.predict_proba(X_test)

In [ ]:
cal_rf_brier = brier_score_loss(y_test, cal_rf_proba[:, 1])

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
CalibrationDisplay.from_estimator(rf, X_test, y_test, ax=ax1, n_bins=10)
ax1.set_title(f"forest without calibration: {rf_brier:0.4f}")
CalibrationDisplay.from_estimator(cal_rf, X_test, y_test, ax=ax2, n_bins=10)
ax2.set_title(f"calibrated: {cal_rf_brier:0.4f}");

### Calibrating the linear model

In [ ]:
inner_lr = make_pipeline(StandardScaler(), LogisticRegression(random_state=42))
cal_lr = CalibratedClassifierCV(inner_lr, method='isotonic')
cal_lr.fit(X_train, y_train)

In [ ]:
cal_lr_proba = cal_lr.predict_proba(X_test)

cal_lr_brier = brier_score_loss(y_test, cal_lr_proba[:, 1])

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
CalibrationDisplay.from_estimator(lr, X_test, y_test, ax=ax1, n_bins=10)
ax1.set_title(f"Logistic without calibration: {lr_brier:0.4f}")
CalibrationDisplay.from_estimator(cal_lr, X_test, y_test, ax=ax2, n_bins=10)
ax2.set_title(f"calibrated: {cal_lr_brier:0.4f}");

## Exercise 2

0. Finish Exercise 1 for training an uncalibrated `GradientBoostingClassifier`.
1. Calibrate the `GradientBoostingClassifier` on the training set. (This can take a little longer)
2. Compute the brier score on the test set.
2. Plot the calibration curve with `n_bins=10` for the calibrated and uncalibrated `GradientBoostingClassifier`, next to each other.
3. Did the calibration improve with `CalibratedClassifierCV`?

**If you are running locally**, you can uncomment the following cell to load the solution into the cell. On **Google Colab**, [see solution here](https://github.com/thomasjpfan/ml-workshop-intermediate-2-of-2/blob/master/notebooks/solutions/02-ex02-solutions.py).

In [ ]:
# %load solutions/02-ex02-solutions.py